## Imports 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import Dropout
from keras import optimizers
import tensorflow as tf 
import keras
import keras.backend as K
from keras.layers.core import Activation
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, LSTM
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
import joblib
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from keras import backend as K

# Nova seção

# Nova seção

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Ingrid"

/content/drive/MyDrive/Ingrid


## Lendo Dados

In [ ]:
df= pd.read_csv('data_05_shaft_bearing.csv')

## Carregando Dados

In [ ]:
df= df.dropna()
df=df[df['power']>100]
df= df[df['outdoor_temp']>0]
df= df[df['nacelle']>0]
df= df[df['generator_speed']>0]
df= df[df['gearbox_bearing']>0]
df.drop(columns=["Unnamed: 0"],inplace=True)
print(df)

features = ['generator_speed', 'nacelle']
target = ['gearbox_bearing']
X = df[features]
y = df[target]

PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)

                      sample_time      power  outdoor_temp  generator_speed  \
0       2018-01-01 00:00:00+00:00  668.28204     32.369460        973.59010   
1       2018-01-01 00:10:00+00:00  396.09955     32.322247        851.24150   
2       2018-01-01 00:20:00+00:00  337.86740     32.230736        835.06190   
3       2018-01-01 00:30:00+00:00  326.90180     32.051624        830.00200   
4       2018-01-01 00:40:00+00:00  400.90637     31.891493        843.18300   
...                           ...        ...           ...              ...   
157809  2020-12-31 21:30:00+00:00  463.78204     30.680513        886.92694   
157810  2020-12-31 21:40:00+00:00  472.37115     30.551275        884.98804   
157811  2020-12-31 21:50:00+00:00  579.70490     30.431310        939.09460   
157812  2020-12-31 22:00:00+00:00  554.78850     30.328987        927.96220   
157813  2020-12-31 22:10:00+00:00  194.68222     30.234390        352.50555   

          nacelle  gearbox_bearing  
0       39.058

## Construindo o Dataset

In [ ]:
X_train= X[0:100000]
y_train= y[0:100000]
X_validation = X[10000:105120]
y_validation = y[10000:105120]
X_test= X[105120:]
y_test= y[105120:]
#print(len(y_test))


#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
#y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1],1))
#X_validation = np.reshape(X_train, (X_validation.shape[0], 1, X_validation.shape[1]))

## Função Plot_Grid

In [ ]:
def Plot_Grid(grid,x,metric,scale_y,namefig):
  sns.set_theme(style="whitegrid")

  plt.rcParams["figure.figsize"] = [scale_y[0], scale_y[1]]
  plt.rcParams["figure.autolayout"] = True

  Grid_List = []

  k_0 = list(grid.keys())[0].split("/")

  del(k_0[-1])

  id_x = ""
  for i in range(len(k_0)):
    if(k_0[i].split(":")[0] == x):
      id_x = i
      break

  combinations = {metric:[],x:[],"hue":[]}

  for key in list(grid.keys()):

    combinations[metric].append(grid[key])

    aux = key.split("/")
    del(aux[-1])

    combinations[x].append(aux[id_x].split(":")[1])

    h = ""
    for j in range(len(aux)):
      if( j!= id_x):
        h += aux[j] + "/"

    combinations["hue"].append(h)



  dataframe = pd.DataFrame(combinations)

  g = sns.catplot(
  data=dataframe, kind="bar",
  x=x, y=metric, hue="hue",
  ci="sd", palette="dark", alpha=.6, height=6
  )
  plt.ylim(scale_y[0], scale_y[1])
  g.despine(left=True)
  g.set_axis_labels(x, metric)
  g.legend.set_title("")
  plt.savefig(namefig + ".png")

    # Alguma forma de ficar setando o tamanho da imagem

## Função R2

In [ ]:
from keras import backend as K

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

## Parâmetros

In [ ]:
grid_name = "LSTM_75"
epochs = [10]
optimizer = ["rmsprop"]
lstm1 = [100]
lstm2 = [400]
batch_size = [60]
activation = ["selu"]
learning_rate = [0.01]
dropout = [0.5]

params = [("batch_size",batch_size), ("epochs",epochs), ("optimizer",optimizer),
          ("lstm1", lstm1), ("lstm2", lstm2), ("activation", activation), ("learning_rate",learning_rate),("dropout",dropout)]

## Model

In [ ]:
def create_model(lstm1,lstm2,activation,optmizer,drop):
Gridsearch
  model = Sequential()
  model.add(LSTM(lstm1,activation=activation, input_shape = (X.shape[1],1),return_sequences=True))
  model.add(Dropout(drop))
  model.add(LSTM(lstm2,activation=activation, return_sequences=True))
  model.add(Dropout(drop))
  model.add(LSTM(lstm2,activation=activation, return_sequences=True))
  model.add(Dropout(drop))
  model.add(LSTM(lstm2,activation=activation, return_sequences=True))
  model.add(Dropout(drop))
  model.add(LSTM(lstm2,activation=activation, return_sequences=False))
  model.add(Dropout(drop))
  model.add(Dense(1, activation=activation))
  model.compile(optimizer=optmizer, loss="mse", metrics=[coeff_determination])
  return model

## GridSearch LSTM

In [ ]:
index = 0
Grid = {}
param_legends = []
all = len(list(itertools.product(batch_size, epochs, lstm1, lstm2, optimizer, activation)))

for pr in params:
  if(len(pr[1])>1):
    param_legends.append(pr[0])

for (bs, ep, l1, l2, opt, ac, lr,dpr) in itertools.product(batch_size, epochs, lstm1, lstm2, optimizer, activation, learning_rate,dropout):

 
  model = create_model(l1,l2,ac,tf.keras.optimizers.RMSprop(learning_rate=lr),dpr)
  model.fit(X_train, y_train , batch_size = bs, epochs= ep)
  results = model.evaluate(X_test, y_test, batch_size= 128)

  key_string = ""

  for pr_lg in param_legends:

    if(pr_lg == "batch_size"):
      key_string = key_string + pr_lg + ":" + str(bs) + "/"

    if(pr_lg == "epochs"):
      key_string = key_string + pr_lg + ":" + str(ep) + "/"

    if(pr_lg == "lstm1"):
      key_string = key_string + pr_lg + ":" + str(l1) + "/"

    if(pr_lg == "lstm2"):
      key_string = key_string + pr_lg + ":" + str(l2) + "/"

    if(pr_lg == "optimizer"):
      key_string = key_string + pr_lg + ":" + str(opt) + "/"

    if(pr_lg == "activation"):
      key_string = key_string + pr_lg + ":" + str(ac) + "/"

    if(pr_lg == "learning_rate"):
      key_string = key_string + pr_lg + ":" + str(lr) + "/"

    if(pr_lg == "dropout"):
      key_string = key_string + pr_lg + ":" + str(dpr) + "/"

  
  Grid[key_string] = results[1]

  percent_process = round(index/all,2)

  if(index % 10 == 0):
    joblib.dump(Grid, grid_name + "_" + "process_" + str(percent_process)+ ".pkl")

  index = index + 1


Epoch 1/10
1667/1667 [==============================] - 43s 23ms/step - loss: 4.0785 - coeff_determination: -3.0857
Epoch 2/10
1667/1667 [==============================] - 38s 23ms/step - loss: 4.0722 - coeff_determination: -3.0809
Epoch 3/10
1667/1667 [==============================] - 38s 23ms/step - loss: 4.0722 - coeff_determination: -3.0873
Epoch 4/10
1667/1667 [==============================] - 39s 23ms/step - loss: 4.0722 - coeff_determination: -3.0618
Epoch 5/10
1667/1667 [==============================] - 38s 23ms/step - loss: 4.0722 - coeff_determination: -3.0993
Epoch 6/10
1667/1667 [==============================] - 38s 23ms/step - loss: 4.0723 - coeff_determination: -3.0745
Epoch 7/10
1667/1667 [==============================] - 38s 23ms/step - loss: 4.0722 - coeff_determination: -3.0739
Epoch 8/10
1667/1667 [==============================] - 38s 23ms/step - loss: 4.0722 - coeff_determination: -3.0867
Epoch 9/10
1667/1667 [==============================] - 38s 23ms/step - 

## Plot Grid

In [ ]:
#Grid_Load = joblib.load("teste_1_process_0.0.pkl")

In [ ]:
Grid

{'': -9.237213134765625}

In [ ]:
Plot_Grid(Grid,"dropout","r2-score",(-1,1),grid_name)

TypeError: ignored